In [45]:
# conécta a mi Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
# Se Crea el dataset de imágenes desde mi google drive

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os

data_dir = '/content/drive/MyDrive/manufactura'

img_size = (224, 224)
batch_size = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

class_names = train_ds.class_names
print("Clases detectadas:", class_names)

Found 44 files belonging to 3 classes.
Using 36 files for training.
Found 44 files belonging to 3 classes.
Using 8 files for validation.
Clases detectadas: ['Candado', 'Destornillador', 'Llave']


In [53]:
# Se Optimiza el flujo de datos
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds   = val_ds.prefetch(buffer_size=AUTOTUNE)

In [54]:
# Define el modelo y determina el numero de clases
num_classes = len(class_names)

base_model = tf.keras.applications.MobileNetV2(
    input_shape=img_size + (3,),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False

data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ]
)

inputs = keras.Input(shape=img_size + (3,))
x = data_augmentation(inputs)
x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)
model = keras.Model(inputs, outputs)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

model.summary()

Model: "functional_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_22 (InputLayer)     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_7 (Sequential)       │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide_7 (TrueDivide)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract_7 (Subtract)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_7      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 3)              │         3,843 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,261,827 (8.63 MB)

 Trainable params: 3,843 (15.01 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [55]:
# Entrenar el modelo

epochs = 15
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.5764 - loss: 0.9542 - val_accuracy: 0.7500 - val_loss: 0.8683
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 464ms/step - accuracy: 0.7025 - loss: 0.7103 - val_accuracy: 0.8750 - val_loss: 0.6174
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 476ms/step - accuracy: 0.9132 - loss: 0.4981 - val_accuracy: 1.0000 - val_loss: 0.4482
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 770ms/step - accuracy: 0.8264 - loss: 0.4406 - val_accuracy: 1.0000 - val_loss: 0.3397
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 460ms/step - accuracy: 0.9711 - loss: 0.2930 - val_accuracy: 1.0000 - val_loss: 0.2693
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 469ms/step - accuracy: 1.0000 - loss: 0.1969 - val_accuracy: 1.0000 - val_loss: 0.2217
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 476ms/step - accuracy: 1.0000 - loss: 0.1546 - val_accuracy: 1.0000 - val_loss: 0.1767
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 705ms/step - accuracy: 1.0000 - loss: 0.1232 - val_accuracy: 1.0000 - val_loss: 0

In [56]:
# Guardar el modelo

model_path = '/content/drive/MyDrive/modelo_herramientas.h5'
model.save(model_path)
print("Modelo guardado en:", model_path)

Modelo guardado en: /content/drive/MyDrive/modelo_herramientas.h5


In [57]:
# Usar el modelo para predecir y probar
import os
import numpy as np
from tensorflow.keras.preprocessing import image

test_dir = '/content/drive/MyDrive/manufactura_ensayo'

for fname in os.listdir(test_dir):
    if not fname.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
        continue

    img_path = os.path.join(test_dir, fname)
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)

    preds = model.predict(img_array)[0]
    predicted_class = class_names[np.argmax(preds)]
    confidence = np.max(preds)

    print(f"{fname} -> {predicted_class} (confianza: {confidence:.2f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
expansion.jpg -> Candado (confianza: 0.69)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
CL-50XL.jpg -> Candado (confianza: 0.68)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
llave-espanola.jpg -> Candado (confianza: 0.70)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
ROTO-1-2A7.jpg -> Candado (confianza: 0.72)
